<a href="https://colab.research.google.com/github/joeyuy/Image-Classification/blob/main/Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

$\rightarrow$ This notebook will train an image classification model imported from keras to identify pokemon by their elemental types. The pokemon image data comes from [Kaggle](https://www.kaggle.com/datasets/vishalsubbiah/pokemon-images-and-types?resource=download&select=images). The model trains itself over 8 trials with randomized parameters selected from predetermined options. The final results of this notebook include an 18 by 18 confusion matrix that shows the model's predictions for the primary type of all pokemon, and a 3 by 3 matrix for the model's predictions on only Fire, Water, and Grass type pokemon.

## Import the necessary libraries

$\rightarrow$ Importing some but not all of the necessary libraries. Some libraries get imported later to combat potential runtime issue problems.

In [1]:
import glob
from PIL import Image
from io import BytesIO
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread
from sklearn.model_selection import train_test_split

## Load in the data

$\rightarrow$ Importing the data from google drive and github.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
files = glob.glob("/content/drive/MyDrive/colab_notebooks/pokemon_images/*.png")

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/joeyuy/Image-Classification/main/pokemon.csv")

In [5]:
df.head()

,Name,Type1,Type2,Evolution
0,bulbasaur,Grass,Poison,ivysaur
1,ivysaur,Grass,Poison,venusaur
2,venusaur,Grass,Poison,NaN
3,charmander,Fire,NaN,charmeleon
4,charmeleon,Fire,NaN,charizard


In [6]:
print(files)

['/content/drive/MyDrive/colab_notebooks/pokemon_images/pawniard.png', '/content/drive/MyDrive/colab_notebooks/pokemon_images/rayquaza.png', '/content/drive/MyDrive/colab_notebooks/pokemon_images/joltik.png', '/content/drive/MyDrive/colab_notebooks/pokemon_images/unfezant.png', '/content/drive/MyDrive/colab_notebooks/pokemon_images/tropius.png', '/content/drive/MyDrive/colab_notebooks/pokemon_images/groudon.png', '/content/drive/MyDrive/colab_notebooks/pokemon_images/electrode.png', '/content/drive/MyDrive/colab_notebooks/pokemon_images/feebas.png', '/content/drive/MyDrive/colab_notebooks/pokemon_images/chimchar.png', '/content/drive/MyDrive/colab_notebooks/pokemon_images/xatu.png', '/content/drive/MyDrive/colab_notebooks/pokemon_images/glalie.png', '/content/drive/MyDrive/colab_notebooks/pokemon_images/heracross.png', '/content/drive/MyDrive/colab_notebooks/pokemon_images/exploud.png', '/content/drive/MyDrive/colab_notebooks/pokemon_images/drampa.png', '/content/drive/MyDrive/colab_no

## Building data lists

$\rightarrow$ Transforming the data into a form that is useful to the model.

In [7]:
X = np.zeros((len(files), 57600))
ytemp = []
for ii, filepath in enumerate(files):
  pokemon_name = filepath.split("/")[-1].split(".png")[0]
  img = imread(filepath)
  X[ii, :] = img.flatten()

  # Extracting characteristics
  type_1 = df.loc[df["Name"] == pokemon_name, "Type1"].values.item()
  ytemp.append(type_1)

In [8]:
ytemp

['Dark',
 'Dragon',
 'Bug',
 'Normal',
 'Grass',
 'Ground',
 'Electric',
 'Water',
 'Fire',
 'Psychic',
 'Ice',
 'Bug',
 'Normal',
 'Normal',
 'Ice',
 'Dark',
 'Dark',
 'Fighting',
 'Electric',
 'Water',
 'Grass',
 'Fighting',
 'Rock',
 'Dragon',
 'Bug',
 'Rock',
 'Bug',
 'Electric',
 'Steel',
 'Electric',
 'Water',
 'Bug',
 'Fire',
 'Rock',
 'Rock',
 'Dragon',
 'Ghost',
 'Water',
 'Electric',
 'Fire',
 'Bug',
 'Grass',
 'Water',
 'Bug',
 'Psychic',
 'Ghost',
 'Dragon',
 'Grass',
 'Bug',
 'Ice',
 'Psychic',
 'Fire',
 'Normal',
 'Normal',
 'Fighting',
 'Normal',
 'Fire',
 'Grass',
 'Water',
 'Water',
 'Water',
 'Fighting',
 'Normal',
 'Ice',
 'Electric',
 'Grass',
 'Psychic',
 'Fire',
 'Water',
 'Water',
 'Normal',
 'Poison',
 'Bug',
 'Bug',
 'Fire',
 'Normal',
 'Fire',
 'Fighting',
 'Rock',
 'Ice',
 'Steel',
 'Electric',
 'Ice',
 'Bug',
 'Rock',
 'Bug',
 'Water',
 'Dragon',
 'Ice',
 'Water',
 'Electric',
 'Fighting',
 'Water',
 'Rock',
 'Dragon',
 'Ice',
 'Bug',
 'Psychic',
 'Normal',


In [9]:
from sklearn.preprocessing import LabelEncoder
y = LabelEncoder().fit_transform(ytemp)

In [10]:
y

array([ 1,  2,  0, 12,  9, 10,  3, 17,  6, 14, 11,  0, 12, 12, 11,  1,  1,
        5,  3, 17,  9,  5, 15,  2,  0, 15,  0,  3, 16,  3, 17,  0,  6, 15,
       15,  2,  8, 17,  3,  6,  0,  9, 17,  0, 14,  8,  2,  9,  0, 11, 14,
        6, 12, 12,  5, 12,  6,  9, 17, 17, 17,  5, 12, 11,  3,  9, 14,  6,
       17, 17, 12, 13,  0,  0,  6, 12,  6,  5, 15, 11, 16,  3, 11,  0, 15,
        0, 17,  2, 11, 17,  3,  5, 17, 15,  2, 11,  0, 14, 12, 14, 10, 17,
        9,  6, 12, 15, 12,  7,  2,  9,  8, 17,  3,  2, 14, 14, 17,  0,  0,
       14, 12, 17, 17,  4, 14, 12, 14,  4, 14,  5,  9,  9, 12, 14,  9,  3,
       17, 12,  9, 10, 16, 13,  1,  4,  6, 14, 12,  9,  8,  1, 13, 10, 14,
       13, 12, 17,  3, 12, 14,  3,  9,  1, 10,  5, 12, 12, 14, 17, 14,  9,
        9, 13, 13,  2, 14, 12, 15,  6,  3, 17,  9,  6, 11,  2,  3,  2, 17,
       16,  0, 12, 14,  3, 10,  6,  0, 17, 17,  3,  3,  0,  0, 13, 11, 16,
       14, 17, 17,  0, 16,  7, 12, 17, 12, 17, 13,  5, 15, 17,  9, 15,  9,
       13, 15, 17,  5, 14

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [12]:
print('X_train: ' + str(X_train.shape))
print('Y_train: ' + str(y_train.shape))
print('X_test:  '  + str(X_test.shape))
print('Y_test:  '  + str(y_test.shape))

X_train: (606, 57600)
Y_train: (606,)
X_test:  (203, 57600)
Y_test:  (203,)


## Normalize Data

$\rightarrow$ Rescaling RGBO values.

In [13]:
print(X_train.max())

255.0


In [14]:
X_train_norm = X_train / 255
X_test_norm = X_test / 255

## Custom Neural Network

$\rightarrow$ Building a neural network that an image classifcation model can use to train data on.

In [15]:
!pip install keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.3 MB/s eta 0:00:00


In [16]:
import keras
import keras_tuner
from keras import layers, regularizers
from keras.optimizers.legacy import Adam
from keras.callbacks import EarlyStopping, TensorBoard

In [17]:
def define_model(units, num_layers, activation, lr, l2):
    model_layers = [
        layers.Dense(units, activation=activation, kernel_regularizer=regularizers.L2(l2=l2), kernel_initializer=keras.initializers.HeNormal())
        ] * num_layers  # These are the main hidden layers
    model_layers += [layers.Dense(18)]  # This is the output layer, which should have the same size as the number of classes you are trying to predict
    model = keras.Sequential(model_layers)  # This just stacks all our layers on top of each other
    model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'], optimizer=Adam(learning_rate=lr))  # This is a good loss/penalty function to use for multi-class classification

    return model

In [18]:
def build_model(hp):
    units = hp.Choice("units", [1024])
    activation = "relu"   # We'll just use ReLu for now
    lr = hp.Float("lr", min_value=1e-5, max_value=1e-3, sampling="log")
    l2 = hp.Float("l2", min_value=1e-5, max_value=1e-3, sampling="log")
    num_layers = hp.Choice("num_layers", [2, 3, 4])

    # call existing model-building code with the hyperparameter values.
    model = define_model(units=units, num_layers=num_layers, activation=activation, lr=lr, l2=l2)
    return model

In [20]:
tuner = keras_tuner.BayesianOptimization(
    hypermodel=build_model,
    objective="val_loss", # We want to see accuracy improve on the validation (or test) dataset
    max_trials=8,  # How many parameter combinations you want to try
    executions_per_trial=1,
    overwrite=False,
    directory="/content/drive/MyDrive/colab_notebooks/poke_sweep_0", # Save the output somewhere you've created
)

In [21]:
# This is a "callback" function, which will get called at the end of every epoch to check whether the accuracy is still increasing
callbacks = [EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True, start_from_epoch=4)]

In [22]:
tuner.search(
    X_train_norm,
    y_train,
    epochs=8,
    verbose=1,
    validation_data=(X_test_norm, y_test),
    callbacks=callbacks
)

Trial 8 Complete [00h 03m 54s]
val_loss: 3.1623222827911377

Best val_loss So Far: 2.851358652114868
Total elapsed time: 00h 25m 17s


In [23]:
# Examining the results
tuner.results_summary(5)  # printing out the top 5 models

Results summary
Results in /content/drive/MyDrive/colab_notebooks/poke_sweep_0/untitled_project
Showing 5 best trials
Objective(name="val_loss", direction="min")

Trial 3 summary
Hyperparameters:
units: 1024
lr: 3.3570553487898416e-05
l2: 1.4699244053052159e-05
num_layers: 2
Score: 2.851358652114868

Trial 4 summary
Hyperparameters:
units: 1024
lr: 3.7359938308603774e-05
l2: 1.5138520101560535e-05
num_layers: 4
Score: 2.885114908218384

Trial 1 summary
Hyperparameters:
units: 1024
lr: 0.00011699898271783844
l2: 1.0865987497113857e-05
num_layers: 2
Score: 2.8877720832824707

Trial 5 summary
Hyperparameters:
units: 1024
lr: 5.159173467296484e-05
l2: 6.023443262491292e-05
num_layers: 4
Score: 2.933765411376953

Trial 0 summary
Hyperparameters:
units: 1024
lr: 3.4205294716664114e-05
l2: 0.0001389647094981114
num_layers: 3
Score: 3.095989942550659


In [26]:
# Best model was number 03. Let's load it in to make a prediction
import json
from sklearn.metrics import confusion_matrix

trial_num = "3"
with open(f"/content/drive/MyDrive/colab_notebooks/poke_sweep_0/untitled_project/trial_{trial_num}/trial.json", "r") as f:
    trial = json.load(f)
hp = trial["hyperparameters"]["values"]
model = define_model(units=hp["units"], num_layers=hp["num_layers"], activation="relu", lr=hp["lr"],
                     l2=hp["l2"])
model.load_weights(f"/content/drive/MyDrive/colab_notebooks/poke_sweep_0/untitled_project/trial_{trial_num}/checkpoint")

In [27]:
y_pred = np.argmax(model.predict(X_test_norm), axis=1)
cm = confusion_matrix(y_test, y_pred)
accuracy = sum(y_pred == y_test) / len(y_test)
print(accuracy)
print(cm)

7/7 [==============================] - 3s 123ms/step
0.15763546798029557
[[ 1  0  0  0  0  0  1  0  5  1  0  3  0  0  0  1  7]
 [ 0  0  0  0  0  0  1  0  1  0  0  3  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  1  0  0  3  0  0  0  0  6]
 [ 1  0  0  0  0  0  0  0  1  0  0  5  0  0  0  0  3]
 [ 1  0  0  0  0  0  0  0  2  0  0  0  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  1  0  0  5  0  0  0  1  1]
 [ 0  1  0  0  0  0  2  0  0  1  0  6  0  0  0  0  4]
 [ 0  0  0  0  0  0  0  0  2  1  0  3  0  0  0  0  3]
 [ 0  0  0  0  0  0  0  0  3  1  0  9  1  0  0  0  4]
 [ 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  1  0  5  0  0  0  0  6]
 [ 3  0  0  0  0  0  0  0  1  1  0 12  0  0  0  0 12]
 [ 1  0  1  0  0  0  0  0  1  0  0  2  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  0  0  5  0  0  0  0  3]
 [ 0  0  0  0  0  0  0  0  1  1  0  7  0  0  0  0  4]
 [ 0  1  0  0  0  0  0  0  4  1  0  2  0  0  0  0  2]
 [ 2  0  1  0  0  0  0  0  2  0  0  7  0  0  0  0 13]]


Woah, 15% is pretty bad. For context, 1/18 is 5.56%, but 15% is just not a useful model at all. Pokemon that have secondary types could be contributing to the model inaccuracy.

## Testing the model on only fire, water, and grass pokemon.

$\rightarrow$ Repeating the process but this time with only fire, water, and grass pokemon.

In [34]:
df.value_counts(subset = 'Type1')

Type1
Water       114
Normal      105
Grass        78
Bug          72
Fire         53
Psychic      53
Rock         46
Electric     40
Poison       34
Ground       32
Fighting     29
Dark         29
Ghost        27
Dragon       27
Steel        26
Ice          23
Fairy        18
Flying        3
Name: count, dtype: int64

In [35]:
114 + 53 + 78

245

In [37]:
X = np.zeros((245, 57600))
ytemp = []
j = 0
for filepath in files:
  pokemon_name = filepath.split("/")[-1].split(".png")[0]
  type_1 = df.loc[df["Name"] == pokemon_name, "Type1"].values.item()
  if type_1 in ["Fire","Water","Grass"]:
    ytemp.append(type_1)
    img = imread(filepath)
    X[j, :] = img.flatten()
    j+=1

In [39]:
y = LabelEncoder().fit_transform(ytemp)

In [42]:
y

array([1, 2, 0, 2, 1, 2, 0, 2, 0, 1, 2, 1, 0, 0, 1, 2, 2, 2, 1, 0, 2, 2,
       0, 0, 2, 2, 2, 2, 1, 0, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 0, 1, 2, 1,
       2, 1, 1, 0, 2, 1, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 0,
       1, 0, 0, 2, 1, 0, 0, 2, 0, 2, 2, 1, 2, 0, 2, 0, 2, 2, 1, 2, 1, 2,
       0, 1, 1, 0, 0, 1, 2, 2, 0, 1, 2, 2, 2, 1, 2, 2, 0, 1, 2, 1, 0, 2,
       2, 2, 1, 2, 2, 0, 0, 1, 2, 2, 2, 2, 1, 2, 1, 0, 2, 0, 1, 2, 1, 1,
       0, 2, 1, 0, 2, 2, 2, 1, 0, 2, 0, 2, 0, 2, 1, 2, 1, 2, 2, 2, 1, 0,
       1, 1, 1, 1, 2, 1, 0, 0, 1, 2, 2, 1, 0, 1, 2, 0, 2, 0, 2, 2, 2, 2,
       2, 2, 1, 2, 1, 0, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 0, 2, 1,
       0, 2, 2, 1, 2, 2, 2, 1, 1, 0, 0, 2, 1, 0, 1, 0, 2, 1, 2, 1, 2, 1,
       1, 2, 2, 1, 0, 2, 2, 2, 2, 0, 2, 1, 1, 1, 1, 2, 2, 1, 2, 2, 0, 2,
       0, 2, 1])

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [41]:
print('X_train: ' + str(X_train.shape))
print('Y_train: ' + str(y_train.shape))
print('X_test:  '  + str(X_test.shape))
print('Y_test:  '  + str(y_test.shape))

X_train: (183, 57600)
Y_train: (183,)
X_test:  (62, 57600)
Y_test:  (62,)


In [43]:
X_train_norm = X_train / 255
X_test_norm = X_test / 255

In [44]:
def define_model_fwg(units, num_layers, activation, lr, l2):
    model_layers = [
        layers.Dense(units, activation=activation, kernel_regularizer=regularizers.L2(l2=l2), kernel_initializer=keras.initializers.HeNormal())
        ] * num_layers  # These are the main hidden layers
    model_layers += [layers.Dense(3)]  # This is the output layer, which should have the same size as the number of classes you are trying to predict
    model = keras.Sequential(model_layers)  # This just stacks all our layers on top of each other
    model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'], optimizer=Adam(learning_rate=lr))  # This is a good loss/penalty function to use for multi-class classification

    return model

In [45]:
def build_model(hp):
    units = hp.Choice("units", [256, 512, 1024])
    activation = "relu"   # We'll just use ReLu for now
    lr = hp.Float("lr", min_value=1e-5, max_value=1e-1, sampling="log")
    l2 = hp.Float("l2", min_value=1e-5, max_value=1e-1, sampling="log")
    num_layers = hp.Choice("num_layers", [2, 3, 4])

    # call existing model-building code with the hyperparameter values.
    model = define_model(units=units, num_layers=num_layers, activation=activation, lr=lr, l2=l2)
    return model

In [46]:
tuner = keras_tuner.BayesianOptimization(
    hypermodel=build_model,
    objective="val_loss", # We want to see accuracy improve on the validation (or test) dataset
    max_trials=8,  # How many parameter combinations you want to try
    executions_per_trial=1,
    overwrite=False,
    directory="/content/drive/MyDrive/colab_notebooks/fwg_sweep_0", # Save the output somewhere you've created
)

In [47]:
# This is a "callback" function, which will get called at the end of every epoch to check whether the accuracy is still increasing
callbacks = [EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True, start_from_epoch=4)]

In [48]:
tuner.search(
    X_train_norm,
    y_train,
    epochs=8,
    verbose=1,
    validation_data=(X_test_norm, y_test),
    callbacks=callbacks
)

Trial 8 Complete [00h 08m 16s]
val_loss: 24.34717559814453

Best val_loss So Far: 0.9370813965797424
Total elapsed time: 00h 22m 35s


In [51]:
tuner.results_summary(5)

Results summary
Results in /content/drive/MyDrive/colab_notebooks/fwg_sweep_0/untitled_project
Showing 5 best trials
Objective(name="val_loss", direction="min")

Trial 5 summary
Hyperparameters:
units: 256
lr: 1.372853907446641e-05
l2: 1.875673879433594e-05
num_layers: 2
Score: 0.9370813965797424

Trial 0 summary
Hyperparameters:
units: 512
lr: 1.616377548778009e-05
l2: 0.00037951310743611957
num_layers: 3
Score: 1.2652881145477295

Trial 3 summary
Hyperparameters:
units: 256
lr: 0.001543252653656617
l2: 0.0011516038886190995
num_layers: 2
Score: 5.097928047180176

Trial 1 summary
Hyperparameters:
units: 1024
lr: 0.00020057166716902651
l2: 0.005296994219666066
num_layers: 2
Score: 5.946966171264648

Trial 6 summary
Hyperparameters:
units: 512
lr: 0.0020574136136957866
l2: 1.0988600943618688e-05
num_layers: 2
Score: 8.011443138122559


In [52]:
# Best model was number 05. Let's load it in to make a prediction
import json
from sklearn.metrics import confusion_matrix

trial_num = "5"
with open(f"/content/drive/MyDrive/colab_notebooks/fwg_sweep_0/untitled_project/trial_{trial_num}/trial.json", "r") as f:
    trial = json.load(f)
hp = trial["hyperparameters"]["values"]
model = define_model(units=hp["units"], num_layers=hp["num_layers"], activation="relu", lr=hp["lr"],
                     l2=hp["l2"])
model.load_weights(f"/content/drive/MyDrive/colab_notebooks/fwg_sweep_0/untitled_project/trial_{trial_num}/checkpoint")

In [53]:
y_pred = np.argmax(model.predict(X_test_norm), axis=1)
cm = confusion_matrix(y_test, y_pred)
accuracy = sum(y_pred == y_test) / len(y_test)
print(accuracy)
print(cm)

2/2 [==============================] - 1s 29ms/step
0.5806451612903226
[[ 6  7  1]
 [ 2  5 13]
 [ 0  3 25]]


Well, 58% is not that good but it's a good enough improvement from 15%.

## Conclusion

Overall, the model struggled to identify pokemon types. This may not be so surprising because pokemon the model relies on colors to identify images. Although there can be correlations between pokemon colors and their types, pokemon designs are highly varied, most of them incorporating multiple dominant colors, especially if they are a dual type pokemon.

Predictably, the model does a much better job when the pool of pokemon is filtered to only include primary types of Fire, Water, and Grass. These colors should be easy to distinguish because the image types are RGBO, but the model results are still slightly underwhelming wiht only 58% overall accuracy.

For further model improvements, the images could be resized to have smaller pixel dimensions so the pokemon's dominant colors are more emphasized. And with smaller parameters, more trials can be ran for the model to train better.